## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

In [39]:
! pip install cassandra-driver

#### Import Packages 

In [1]:
import pandas as pd
import numpy as np
import cassandra
import csv
df = pd.read_csv('event_data.csv')
df

,artist_name,fname,gender,item_in_session_number,lname,length,level,location,session_number,song_title,user_id
0,Pavement,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",345,Mercy:The Laundromat,10
1,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",438,Horn Concerto No. 4 in E flat K495: II. Romanc...,53
2,Gary Allan,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",438,Nothing On But The Radio,53
3,Charttraxx Karaoke,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",438,Fireflies,53
4,The Libertines,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",389,The Good Old Days,29
...,...,...,...,...,...,...,...,...,...,...,...
6815,Aventura,Jacob,M,50,Klein,278.04689,paid,"Tampa-St. Petersburg-Clearwater, FL",954,Peligro,73
6816,Die Sekte,Layla,F,86,Griffin,206.18404,paid,"Lake Havasu City-Kingman, AZ",984,Mittelfinga ab feat. Sido_ MOK_ B-Tight_ Bendt,24
6817,Kid Dynamite,Jacob,M,51,Klein,119.95383,paid,"Tampa-St. Petersburg-Clearwater, FL",954,Never Met The Gooch,73
6818,Eminem / Bizarre,Jacob,M,52,Klein,244.55791,paid,"Tampa-St. Petersburg-Clearwater, FL",954,Amityville,73


In [2]:
df.head()

,artist_name,fname,gender,item_in_session_number,lname,length,level,location,session_number,song_title,user_id
0,Pavement,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",345,Mercy:The Laundromat,10
1,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",438,Horn Concerto No. 4 in E flat K495: II. Romanc...,53
2,Gary Allan,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",438,Nothing On But The Radio,53
3,Charttraxx Karaoke,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",438,Fireflies,53
4,The Libertines,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",389,The Good Old Days,29


In [3]:
df.shape

(6820, 11)

In [4]:
df.dtypes

artist_name                object
fname                      object
gender                     object
item_in_session_number      int64
lname                      object
length                    float64
level                      object
location                   object
session_number              int64
song_title                 object
user_id                     int64
dtype: object


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [5]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# create a session to establish connection and begin executing queries

In [6]:

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)
 

#### Create & Set Keyspace

In [7]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS nosqldb WITH REPLICATION = 
                        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
   
    session.set_keyspace("nosqldb")

except Exception as e:
    print(e)

# List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [8]:
## Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
## that was heard during session_number = 338, and item_in_session_number = 4
## make use of create table command  

query = "CREATE TABLE IF NOT EXISTS session_table "

query = query + "(artist_name text, song_title varchar, length float, session_number int, item_in_session_number int, PRIMARY KEY (session_number, item_in_session_number))"
try:
    session.execute(query)
except Exception as e:
   
    print(e)
    


### Let's insert our data into of table

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO session_table (artist_name, song_title, length, session_number, item_in_session_number)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (row[0], row[9], float(row[5]), int(row[8]), int(row[3])))

### Validate our Data Model using a SELECT

In [10]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "SELECT * FROM session_table WHERE session_number = 338 and item_in_session_number = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist_name, row.song_title, row.length, row.session_number, row.item_in_session_number)
    

Faithless Music Matters (Mark Knight Dub) 495.30731201171875 338 4


### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [11]:
## Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
## name(fname and lname) of the user for user_id = 10, session_number = 182 
## make use of create table command
query = "CREATE TABLE IF NOT EXISTS user_table"
query = query + "(artist_name text, fname text, lname text,song_title varchar, session_number int, user_id int, item_in_session_number int, PRIMARY KEY ((user_id, session_number), item_in_session_number, fname, lname)) WITH CLUSTERING ORDER BY (item_in_session_number ASC, fname ASC, lname ASC);"
try:
    session.execute(query)
except Exception as e:
    print(e)


### Let's insert our data into of table

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query =   "INSERT INTO user_table (artist_name, fname, lname, song_title, session_number, user_id, item_in_session_number)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (row[0], row[1], row[4], row[9], int(row[8]), int(row[10]), int(row[3])))

### Validate our Data Model using a SELECT

In [13]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results

query = "SELECT * FROM user_table WHERE session_number = 182 and user_id = 10 ALLOW FILTERING"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.fname, row.lname, row.song_title, row.session_number, row.user_id, row.item_in_session_number)
    

Down To The Bone Sylvie Cruz Keep On Keepin' On 182 10 0
Three Drives Sylvie Cruz Greece 2000 182 10 1
Sebastien Tellier Sylvie Cruz Kilometer 182 10 2
Lonnie Gordon Sylvie Cruz Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 182 10 3


### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [15]:
## Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
## to the song_title 'All Hands Against His Own'
## make use of create table command
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results

query = "CREATE TABLE IF NOT EXISTS user_song_table"
query = query + "(fname text, lname text, song_title varchar, user_id int, PRIMARY KEY(user_id, song_title))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Let's insert our data into of table

In [17]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO user_song_table (fname, lname, song_title, user_id)"
        query = query + "VALUES (%s, %s, %s, %s)"
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (row[1], row[4], row[9], int(row[10])))

### Validate our Data Model using a SELECT

In [18]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "SELECT * FROM user_song_table WHERE song_title = 'All Hands Against His Own' ALLOW FILTERING"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.fname, row.lname, row.song_title)

Sara Johnson All Hands Against His Own
Tegan Levine All Hands Against His Own
Jacqueline Lynch All Hands Against His Own


### Drop the tables before closing out the sessions

In [19]:
query = "drop table user_table"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_table"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_table"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)    
    

Error from server: code=2200 [Invalid query] message="unconfigured table user_table"
Error from server: code=2200 [Invalid query] message="unconfigured table song_table"


### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()